In [1]:
import os
from pyspark.sql import SparkSession

jars = []
for a,b,c in os.walk("/opt/bitnami/spark/jars"):
    for jar in c:
        jars.append(os.path.join(a,jar))
jars = ",".join(jars)

spark = (
    SparkSession.builder
    .appName("IcebergExample") 
    .config("spark.jars", jars)
    .master("spark://spark-master:7077")
    .config("spark.sql.catalog.hadoop_cat", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.hadoop_cat.type", "hive")
    .config("spark.sql.catalog.hadoop_cat.uri", "thrift://metastore:9083")
    .config("spark.sql.catalog.hadoop_cat.warehouse", f"s3a://{os.environ['WAREHOUSE_BUCKET_NAME']}/warehouse")
    .getOrCreate()
)

# Read the Parquet file from MinIO bucket "source"
df = spark.read.parquet("s3a://raw/sample.parquet")

df.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
+---+-------+



In [2]:
spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

'3.3.4'

In [2]:
spark.sql("CREATE DATABASE IF NOT EXISTS hadoop_cat.db")

DataFrame[]

In [4]:
# Write to Iceberg table in the "warehouse" bucket
df.writeTo("hadoop_cat.db.people").using("iceberg").createOrReplace()